In [165]:
# == importar bibliotecas == 

from bs4 import BeautifulSoup as bs
import unicodedata as ucd
import requests
import pandas as pd
import numpy as np

In [238]:
# == settando variáveis == 

scrap = pd.DataFrame()
JORNAL = 'PSOL'
LABEL = 'esquerda' 
URL = "https://psol50.org.br/noticias/"
class_1 = 'entry-title td-module-title'
class_2 = 'td_block_wrap tdb_single_content tdi_75 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'

In [126]:
# == obter html == 

def get_soup(url):
    response = requests.get(url)
    html = response.content
    soup = bs(html, 'lxml')
    return soup

In [234]:
# == regra de restrição na seleção == 

def check(text):
    restricoes = ['<a', '<href']
    if any(map(text.__contains__, restricoes)):
        return False
    return True

In [248]:
# == obter dataframe do texto == 

def return_df(url, text):
    return pd.DataFrame({
        'jornal': JORNAL,
        'url': url,
        'artigo': text,
        'label': LABEL
    }, index=[1])

In [127]:
# == lista de URLs ==

soup = get_soup(URL)
list_url = []

In [167]:
# == coletar os sites de artigos == 

for noticia in soup.find_all("h3", class_=class_1):
    for artigo in noticia.find_all("a", href=True):
        list_url.append(artigo.get('href'))

In [249]:
# == obter conteúdo da notícia == 

for url in list_url:
    soup = get_soup(url)
    list_text = []
    for div in soup.find_all("div", class_=class_2):
        for p in div.find_all("p"):
            if check(str(p)):
                list_text.append(ucd.normalize("NFKD", p.text))
        scrap = pd.concat([
            scrap,
            return_df(url, " ".join(list_text))
        ])
        
scrap


,jornal,url,artigo,label
1,PSOL,https://psol50.org.br/com-relatorio-de-tarcisi...,A Comissão de Constituição e Justiça da Ca...,esquerda
1,PSOL,https://psol50.org.br/psol-aciona-ministerio-p...,"Os deputados federais do PSOL Erika Hilton, Pa...",esquerda
1,PSOL,https://psol50.org.br/nota-do-psol-pt-e-pcdob-...,Editorial do Estadão do dia 13 de outubro ata...,esquerda
1,PSOL,https://psol50.org.br/erika-hilton-aciona-cnj-...,A deputada federal Erika Hilton (PSOL) apresen...,esquerda
1,PSOL,https://psol50.org.br/psol-lamenta-mortes-e-co...,O apartheid sionista de Israel vem empobrecend...,esquerda
1,PSOL,https://psol50.org.br/nota-do-psol-nalu-faria-...,Fomos surpreendidos com a triste notícia de q...,esquerda
1,PSOL,https://psol50.org.br/nota-de-pesar-do-psol/,O Partido Socialismo e Liberdade (PSOL) manife...,esquerda
1,PSOL,https://psol50.org.br/psol-e-pt-acionam-stf-pa...,"O PSOL, ao lado do PT, protocolou na última s...",esquerda
1,PSOL,https://psol50.org.br/psol-democratico-forte-e...,O 8o Congresso Nacional do PSOL aprovou uma re...,esquerda
1,PSOL,https://psol50.org.br/transformar-as-cidades-e...,"Na tarde deste sábado (30), o 8o Congresso Na...",esquerda


In [250]:
# == salvar o scrapping == 

scrap.to_parquet('../dataset/scrapping/PSOL.parquet')

In [251]:
scrap.arti

,jornal,url,artigo,label
1,PSOL,https://psol50.org.br/com-relatorio-de-tarcisi...,A Comissão de Constituição e Justiça da Ca...,esquerda
1,PSOL,https://psol50.org.br/psol-aciona-ministerio-p...,"Os deputados federais do PSOL Erika Hilton, Pa...",esquerda
1,PSOL,https://psol50.org.br/nota-do-psol-pt-e-pcdob-...,Editorial do Estadão do dia 13 de outubro ata...,esquerda
1,PSOL,https://psol50.org.br/erika-hilton-aciona-cnj-...,A deputada federal Erika Hilton (PSOL) apresen...,esquerda
1,PSOL,https://psol50.org.br/psol-lamenta-mortes-e-co...,O apartheid sionista de Israel vem empobrecend...,esquerda
1,PSOL,https://psol50.org.br/nota-do-psol-nalu-faria-...,Fomos surpreendidos com a triste notícia de q...,esquerda
1,PSOL,https://psol50.org.br/nota-de-pesar-do-psol/,O Partido Socialismo e Liberdade (PSOL) manife...,esquerda
1,PSOL,https://psol50.org.br/psol-e-pt-acionam-stf-pa...,"O PSOL, ao lado do PT, protocolou na última s...",esquerda
1,PSOL,https://psol50.org.br/psol-democratico-forte-e...,O 8o Congresso Nacional do PSOL aprovou uma re...,esquerda
1,PSOL,https://psol50.org.br/transformar-as-cidades-e...,"Na tarde deste sábado (30), o 8o Congresso Na...",esquerda
